# Notebook trabajo Aquiles Technologies



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!usr/bin/bash python
!pip install opencv-python-headless
!pip install opencv-contrib-python
!pip install --upgrade imutils

In [ ]:
import cv2
import numpy as np
import time, pandas
import imutils
import imageio
import time
import matplotlib.animation as animation

from google.colab import output
from scipy.spatial import distance as dist
from datetime import datetime
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from imutils.video import VideoStream
from imutils.video import FPS
from IPython.display import Image
from skimage.transform import resize
from IPython.display import HTML

In [ ]:

(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
print(cv2.__version__)
print(major_ver)
print(minor_ver)
print(subminor_ver)

if __name__ == '__main__' :

    # Set up tracker. Instead of CSRT, you can also use
    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    tracker_type = tracker_types[7]

    if int(minor_ver) < 3:
        tracker = cv2.Tracker_create(tracker_type)
    else:
        if tracker_type == 'BOOSTING':
            tracker = cv2.TrackerBoosting_create()
        elif tracker_type == 'MIL':
            tracker = cv2.TrackerMIL_create()
        elif tracker_type == 'KCF':
            tracker = cv2.TrackerKCF_create()
        elif tracker_type == 'TLD':
            tracker = cv2.TrackerTLD_create()
        elif tracker_type == 'MEDIANFLOW':
            tracker = cv2.TrackerMedianFlow_create()
        elif tracker_type == 'GOTURN':
             tracker = cv2.TrackerGOTURN_create()
        elif tracker_type == 'MOSSE':
            tracker = cv2.TrackerMOSSE_create()
        elif tracker_type == "CSRT":
            tracker = cv2.TrackerCSRT_create()

tracker_type


In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Notebooks proyectos/PROYECTO AQUILES/PRUEBA TÉCNICA - AQUILES SOLUTIONS/Input.mp4')

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps  = cap.get(cv2.CAP_PROP_FPS)
fcount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print("WIDTH: '{}'".format(width))
print("HEIGHT : '{}'".format(height))
print("FPS : '{}'".format(fps))
print("FRAME_COUNT  : '{}'".format(fcount))
print("CAP_PROP_CONVERT_RGB : '{}'".format(cap.get(cv2.CAP_PROP_CONVERT_RGB)))
print("CAP_PROP_POS_MSEC : '{}'".format(cap.get(cv2.CAP_PROP_POS_MSEC)))
print("CAP_PROP_BRIGHTNESS : '{}'".format(cap.get(cv2.CAP_PROP_BRIGHTNESS)))
print("CAP_PROP_CONTRAST : '{}'".format(cap.get(cv2.CAP_PROP_CONTRAST)))
print("CAP_PROP_SATURATION : '{}'".format(cap.get(cv2.CAP_PROP_SATURATION)))
print("CAP_PROP_HUE : '{}'".format(cap.get(cv2.CAP_PROP_HUE)))
print("CAP_PROP_GAIN  : '{}'".format(cap.get(cv2.CAP_PROP_GAIN)))


# release window
cap.release()
cv2.destroyAllWindows()

In [ ]:
def display_video(video):
    fig = plt.figure(figsize=(3,3))  #Display size specification

    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=17)

    plt.close()
    return anime

video = imageio.mimread('/content/drive/MyDrive/Notebooks proyectos/PROYECTO AQUILES/PRUEBA TÉCNICA - AQUILES SOLUTIONS/Input.mp4', memtest=False)
video = [resize(frame, (512, 512))[..., :3] for frame in video]    #Size adjustment (if necessary)
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5


In [ ]:

cap = cv2.VideoCapture('/content/drive/MyDrive/Notebooks proyectos/PROYECTO AQUILES/PRUEBA TÉCNICA - AQUILES SOLUTIONS/Input.mp4')

ret, frame1 = cap.read()
ret, frame2 = cap.read()

print(frame1.shape)
print(ret)

fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
out = cv2.VideoWriter("/content/drive/MyDrive/Notebooks proyectos/PROYECTO AQUILES/PRUEBA TÉCNICA - AQUILES SOLUTIONS/output.mp4", fourcc, 60, (1280,720))

x1 = []
y1 = []
time=[]
t = 0
n = 0

while n < 166:

    t = t + 0.017
    print('frame diferencia numero = ', n)

    diff = cv2.absdiff(frame1, frame2)

    print('ret = ', ret)
    print('shape frame1 = ', frame1.shape)
    print('shape frame2 = ', frame2.shape)
    print('shape diff = ', diff.shape)


    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 200:  # 900
            continue
        if len(contours)>1:
            contour = max(contours, key=cv2.contourArea)

    if n > 3:
       cv2.rectangle(diff, (x, y), (x+w, y+h), (0, 255, 0), 3)
       cv2.putText(diff, "Status: {}".format('Movement'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
       print(x, y, w, h)
       x1.append(np.float16(x))
       y1.append(np.float16(y))
       time.append(np.float16(t))
       out.write(diff)

    plt.title("diff")
    plt.imshow(diff)
    plt.show()

    frame1 = frame2
    ret, frame2 = cap.read()
    n = n + 1

    if cv2.waitKey(40) == 27:
        break



else:
    print('END')
    cv2.destroyAllWindows()

cap.release()
out.release()

In [ ]:

fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection="3d")

# Data for a three-dimensional line
zline = time
xline = x1
yline = y1

ax.set_title('Tracking 3D trajectory');
ax.plot3D(xline, yline, zline, "hotpink")
plt.show()

# **END**